In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import time
import os
from IPython.display import clear_output

## Agenda

- look up how to simulate cellular automata 
- how to incorporate temporal delays
- how to apply that on graphs

In [ ]:
class Neuron(object):
    def __init__(self, _input):
        
    def refractory_period(self):
        
        
class Network(Neuron):
    def __init__(self, Neurons):
        Neuron.__init__(self, _input)
        

### Noodling with cellular automata

1d case:

In [341]:
d1_n = np.zeros(5)

def update(neurons):
    for i, n in enumerate(neurons):
        if neurons[i] == 1:
            try:
                neurons[i] = 0
                neurons[i+1] = 1
                break
            except IndexError:
                neurons[i] = 0
    return neurons
    
    
def pulse(neurons):
    neurons[0] = 1
    for i in neurons:
        print(np.array(neurons), end="\r")
        update(neurons)
        time.sleep(0.5)
    print(neurons, end="\r")

In [342]:
pulse(np.zeros(5, dtype = int))

### 2D case:

In [387]:
def update2d(neurons):
    for i in range(len(neurons)): #where d1 is 1d array of neurons in neurons[:]
        for j in range(len(neurons[i])):
            if neurons[i][j] == 1:
                try:
                    neurons[i][j] = 0
                    neurons[i+1][j+1] = 1
                    #neurons[i-1][j-1] = 1
                    return neurons
                except IndexError:
                    neurons[i][j] = 0
                    return neurons
    
def pulse2d(neurons):
    #neurons[0] = 1
    while 1 in np.matrix(neurons):
        print(np.matrix(neurons), end="\r")
        update2d(neurons)
        time.sleep(.3)
        clear_output(wait=True)
    print(np.matrix(neurons), end="\r")

In [388]:
d2 = [list(np.zeros(10, dtype= int)) for i in np.arange(10)]
d2[5][5] = 1
d2

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [389]:
pulse2d(d2)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
